# 2D Lung Segmentation for Deeblabv1

According to the World Heath Organization, lung cancer is the leading cause of cancer-related deaths worldwide, accounting for an estimated 1.4 million deaths in 2018 (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3864624/). CT scans are now being used in the United States to screen high-risk individuals for lung cancer. In order to detect cancerous lesions in scans, a useful first step is to segment the lungs in the image. (Then a researcher would use just the segmented image as input to another model). In this notebook we will perform segmentation on lungs in CT scans using a U-Net model (https://sites.pitt.edu/~sjh95/related_papers/u-net.pdf).


In [ ]:
%env TF_CPP_MIN_LOG_LEVEL=3  # silence some TensorFlow warnings and logs.

import os
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Please feel free to play with hyperparameters, but submit the assignment with the original values.
IMAGE_SIZE = (128, 128)
TRAIN_VAL_TEST_SPLIT = (0.6, 0.2, 0.2)
BATCH_SIZE = 16
SEED = 220
EPOCHS = 40

## Section 1: Data

We will be using a Kaggle dataset from https://www.kaggle.com/kmader/finding-lungs-in-ct-data. Similar to part 1, create an API token to set in your environment and download the dataset from Kaggle through the following commands.

```kaggle datasets download -d kmader/finding-lungs-in-ct-data```

```unzip finding-lungs-in-ct-data.zip```

This should take a couple of minutes. Fill IMAGE_DIR and MASK_DIR with the output paths.

In [ ]:
# FILL IN CODE HERE #

IMAGE_DIR = '' # replace with your path
MASK_DIR = '' # replace with your path
PROJECT_DIR = ''

# FILL IN CODE HERE #

**Q1b.1**: First let's load the dataset from the saved folders. Our dataset contains 267 CT scans and the corresponding segmentations. Implement the ```load_data``` function below.

For each file (CT scan or mask):

1) Read the image using ```cv2.imread```. Make sure to read all channels with the ```cv2.IMREAD_UNCHANGED``` argument.

2) Normalize the image by its maximum and minmum value, so each image will be in the range [0,1].

3) For the segmentation mask only, normalize the mask by its maximum value and convert the numpy array to type `int16`.

4) Resize the image to ```IMAGE_SIZE``` using the ```cv2.resize``` function. For images, use the ```cv2.INTER_LANCZOS4``` interpolator, and for segmentation masks use the `INTER_NEAREST` interpolator.


Then split all the data into three training (60% of data), validation (20% of data), and test (20% of data). The expected shapes of the ```train_images``` and ```train_masks``` arrays are (160, 128, 128, 1).

In [ ]:
import random

def load_data(IMAGE_DIR, MASK_DIR):
    """ Load images and mask from the saved folders, IMG_DIR and MASK_dir and split them to
        train, validation, and test sets.

        Input:
            IMAGE_DIR (str): path to the folder containing ct scan images
            MASK_DIR (str): path to the folder containing corresponding masks
        Output:
            train_images, val_images, test_images (numpy array of shape (number of images, IMAGE_SIZE)):
                preprocessed images split in 0.6, 0.2, 0.2 respectively
            train_masks, val_masks, test_masks (numpy array of shape (number of masks, IMAGE_SIZE)):
                preprocessed masks split in 0.6, 0.2, 0.2 respectively
    """
    images = []
    masks = []
    for i, file in enumerate(sorted(os.listdir(IMAGE_DIR))):
        image_path = os.path.join(IMAGE_DIR, file)
        mask_path = os.path.join(MASK_DIR, file)

        # FILL IN CODE HERE #

        # FILL IN CODE HERE #

    all_images = np.stack(images)[:,:,:,np.newaxis]
    all_masks = np.stack(masks)[:,:,:,np.newaxis]

    # FILL IN CODE HERE #

    # FILL IN CODE HERE #

    return train_images, train_masks, val_images, val_masks, test_images, test_masks

train_images, train_masks, val_images, val_masks, test_images, test_masks = load_data(IMAGE_DIR, MASK_DIR)
print('Train images: ', train_images.shape)
print('Train masks: ', train_masks.shape)

**Q1b.2**: We will now implement data augmentation. As we can see from the previous part, we only have 160 training examples. Neural networks usually need large number of training examples to succeed in learning a task such as segmentation. Therefore, we will use data augmentation techniques to simulate exposing our model to many more training examples. We will use the following augmentations on our data:

1) Random horizontal shift with at most 0.1 of image width

2) Random vertical shift with at most 0.1 of image height

3) Random rotation with at most 10 degrees

4) Random zoom with at most 0.1 times zoom-in or zoom-out

Explain why these augmentations should help improve the accuracy of the model. Is it possible for data augmentation to instead decrease the performance of the trained model on the test set? If so, explain under what situation this could happen.

*Written answer*:



**Q1b.3**: For data augmentation in Keras, we will use the ```ImageDataGenerator``` class which has several built-in data augmentation methods. Look at the documentation of [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) and implement the following function. You only need to define `image_datagen` and `mask_datagen`.

The final print statement will check that your data generator is returning data with the right shape.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(train_images, train_masks):
    """ Creat a generator based on training images and masks.

        Input:
            train_images (np): array of training images
            train_masks (np): array of training masks
        Output:
            data_generator (generator function): generator which yields images and masks after data augmenetation
    """
    # FILL IN CODE HERE #
    # Define `image_datagen` and `mask_datagen`
    # FILL IN CODE HERE #

    image_datagen.fit(train_images, augment=True, seed=SEED)
    mask_datagen.fit(train_masks, augment=True, seed=SEED)

    image_generator = image_datagen.flow(train_images, batch_size=BATCH_SIZE, seed=SEED)
    mask_generator = mask_datagen.flow(train_masks, batch_size=BATCH_SIZE, seed=SEED)
    data_generator = zip(image_generator, mask_generator)


    return data_generator

data_generator = create_generator(train_images, train_masks)
img, mask = next(data_generator)
print(f"Image batch shape {img.shape}, mask batch shape {mask.shape}")

Run the following code to convert the generator (for the training set) and numpy arrays (for the validation and test sets) to TensorFlow datasets. Don't worry about the details; in part 3 of this assignment you will get practice writing generators for TensorFlow datasets.

In [ ]:
def train_generator():
    while True:
        image_batch, mask_batch = next(data_generator)
        yield image_batch, mask_batch

train_dataset = tf.data.Dataset.from_generator(train_generator,
                     output_types=(tf.float32, tf.float32),
                     output_shapes=([None, IMAGE_SIZE[0], IMAGE_SIZE[1], 1],
                                    [None, IMAGE_SIZE[0], IMAGE_SIZE[1], 1])
                                              ).repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks)).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_masks)).batch(BATCH_SIZE)

**Q1b.4**: Let's take a look at some image examples and their corresponding masks from the training dataset. Take a batch from the ```train_dataset``` and plot four images and their corresponding masks.

In [ ]:
# FILL IN CODE HERE #

# FILL IN CODE HERE #

## Section 2: Model

Let's implement the Deeplab v1 model for our segmentation task. You can see the model architecture described in the Deeplab paper here: (arxiv.org/abs/1412.7062) , Figure 1. We'll be implementing a simpler version of this without a weighted loss for class imbalance.

**Q1b.5**: At a high level, explain how U-Net works and why it is more suitable for the segmentation task relative to a simple convolutional neural network (CNN).


*Written answer*:

**Q1b.6**: Implement a simplified version of Deeplab based on thesis.


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, Dropout, concatenate
class Deeplabv1(tf.keras.Model):
    def __init__(self):
        super(Deeplabv3, self).__init__()

        # FILL IN CODE HERE #

        # FILL IN CODE HERE #

    def call(self, inputs):
        # FILL IN CODE HERE #

        # FILL IN CODE HERE #

        return output

# create model
model = Deeplabv1()

# test that the output data shape is reasonable
img_batch, mask_batch = next(iter(test_dataset))
sample_out = model(img_batch)
print(f"Input shape {img_batch.shape}, model out shape {sample_out.shape}")

**Q1b.7**: Which loss function do you think is most appropriate for our task here? Why? After selecting the loss function, compile the model using the Adam optimizer with default learning rate. Add the accuracy metric.

(Note that the UNet paper describes a special weighting coefficient that makes its loss function different to the one you may choose; you don't have to implement this weighting.)

*Written answer*:

In [ ]:
# FILL IN CODE HERE #
model.compile(

)
# FILL IN CODE HERE #

**Q1b.8**: Before training the model, let's first set a learning rate scheduler. This will adjust the learning rate during the training of the model. Why can it be useful to use a learning rate schedule to train complex neural networks?

*Written answer*:

Now we are going to train our model for 40 epochs. Implement the following function such that the learning rate for the first 20 epochs is 0.001, and after that it decreases exponentially with a factor of 0.1. For more information, please look at the ```ExponentialDecay``` documentation in Keras.

In [ ]:
def scheduler(epoch):
    """ Schedule the learning rate to be 0.0001 for first
        20 epoch, and decrease exponentially by a factor of 0.1
        for all remaining epochs.

        Input:
            epoch (int)
        Output:
            learning_rate (float)
    """
    # FILL IN CODE HERE #

    # FILL IN CODE HERE #

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)


**Q1b.9**: Train the model using the ```model.fit``` function in the Keras API. We compute the value of `steps_per_epoch` which tells the API how many times to sample from the TensorFlow dataset object. It should be passed as an argument to `model.fit()`. To enable the learning rate schedule we just implemented, pass it as a callback to `model.fit`.

In [ ]:
steps_per_epoch = int(len(os.listdir(IMAGE_DIR)) * TRAIN_VAL_TEST_SPLIT[0]) // BATCH_SIZE # pass this arg to fit()


hist = model.fit(
    # FILL IN CODE HERE #

    # FILL IN CODE HERE #
)

Using the `hist` variable returned from running the `fit` command above, run the following code to plot the training and validation loss.

In [ ]:
plt.figure()
plt.plot(range(EPOCHS), hist.history['loss'], label='Training loss')
plt.plot(range(EPOCHS), hist.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()
plt.show()

## Section 3: Evaluation

**Q1b.10**: Use ```model.evaluate``` to evaluate the accuracy of the model on the test set. You should have an accuracy greater than 0.9.

In [ ]:
model.evaluate(test_dataset)


**Q1b.11**: Based on the loss and accuracy on the test set, do you think we are overfitting? Please describe three methods that can be used to avoid overfitting.

*Written answer*:

**Q1b.12**: What is Intersection over Union (IoU) and why it is a useful metric for the segmentation task?
Implement the following function using the definition of IoU. Note that the ```np.logical_and``` and ```np.logical_or``` could be useful.

*Written answer*:

In [ ]:
def compute_IoU(target, prediction):
    """
    Evaluate the intersection over union score for a single prediction and ground truth value

    Parameters:
    target: (np.ndarray) : The ground truth label values
    prediction (np.ndarray): The labels predicted by the model
    """
    # FILL IN CODE HERE #

    # FILL IN CODE HERE #
    return iou

**Q1b.13**: Using the `compute_IoU` function, compute the mean IoU on the test set and print the result. Compute the IoU for each image separately, and then take the mean over images. You can binarize the prediction mask with a threshold of 0.1.

In [ ]:
# FILL IN CODE HERE #

# FILL IN CODE HERE #

**Q1b.14**: Let's make some predictions! Plot four test images, their corresponding ground truth mask, and the model prediction. Make sure to show at least one image for each of the IoU ranges of [0.85, 0.9], [0.9, 0.95], and [0.95, 1].

In [ ]:
# FILL IN CODE HERE #


# FILL IN CODE HERE #